In [11]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import os
import time
import json
def tic():
    return time.time()
def toc(starttime):
    return time.time()-starttime
model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 319.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [41]:
PATH="youcookii_annotations_trainval.json"
counter=0
if not os.path.exists(PATH):
    !wget http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
    !tar -xvzf youcookii_annotations_trainval.tar.gz
    with open('youcookii_annotations_trainval.json', 'r') as file:
        data = json.load(file)
        data = data['database']
else:
    with open('progress.txt', 'r') as file:
        counter=int(file.read())
    with open('youcookii_annotations_trainval.json', 'r') as file:
        data = json.load(file)

--2024-12-09 12:16:34--  http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
Resolving youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)... 141.212.115.67
Connecting to youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)|141.212.115.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337048 (329K) [application/octet-stream]
Saving to: ‘youcookii_annotations_trainval.tar.gz’

youcookii_annotatio 100%[===================>] 329.15K  2.08MB/s    in 0.2s    

2024-12-09 12:16:34 (2.08 MB/s) - ‘youcookii_annotations_trainval.tar.gz’ saved [337048/337048]

youcookii_annotations_trainval.json


In [42]:
counter

0

In [23]:
sentences = []
for item in data.values():
  annotations = item['annotations']
  for annotation in annotations:
    sentences.append(annotation['sentence'])

In [24]:
for i in range(10):
  print(sentences[i])

spread margarine on two slices of white bread
place a slice of cheese on the bread
place the bread slices on top of each other and place in a hot pan
flip the sandwich over and press down
cut the sandwich in half diagonally
pick the ends off the verdalago
combine lemon juice sumac garlic salt and oil in a bowl
chop lettuce and place it in a bowl
add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
pour the dressing over the salad and mix


In [25]:
def cache(translations,counter):
    with open('Translations.txt', 'a') as file:
      for string in translations:
        file.write(string + "\n")
    with open('progress.txt', 'w') as file:
        file.write(f"{counter}")

In [26]:
def add_translation(trans):
    j = 0
    for key in data.keys():
        item = data[key]
        annotations = item['annotations']
        for i in range(len(annotations)):
            if j == len(trans):
                break
            print(i)
            data[key]['annotations'][i]['translation'] = trans[j]
            j+=1

In [59]:
def run_model_batch(input_strings, batch_size=8,max_new_tokens=50,initial_index=0, **generator_args):
    translations = []
    counter=initial_index
    start=tic()
    trans_check=initial_index
    for i in range(initial_index, len(input_strings), batch_size):
        batch = input_strings[i:i + batch_size+1]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        res = model.generate(input_ids, attention_mask=attention_mask,max_new_tokens=max_new_tokens, **generator_args)
        outputs = tokenizer.batch_decode(res, skip_special_tokens=True)
        translations.extend(outputs)
        counter+=batch_size
        if(toc(start)>=60):
            start=tic()
            cache(translations[trans_check:],counter)
            trans_check=i+batch_size

    return translations


In [60]:
trans = run_model_batch(sentences[:80],initial_index=counter)

KeyboardInterrupt: 

In [45]:
counter

0

In [49]:
for i in range(80):
  print(f"{i+1}-"+sentences[i])

1-spread margarine on two slices of white bread
2-place a slice of cheese on the bread
3-place the bread slices on top of each other and place in a hot pan
4-flip the sandwich over and press down
5-cut the sandwich in half diagonally
6-pick the ends off the verdalago
7-combine lemon juice sumac garlic salt and oil in a bowl
8-chop lettuce and place it in a bowl
9-add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
10-pour the dressing over the salad and mix
11-add the fried pita to the salad and mix
12-combine salt butter milk and hot sauce in a bowl
13-place chicken in a small bowl and pour brine over chicken and freeze it for 1 hour
14-pour off the brine and rinse chicken off thoroughly and shake off excess water
15-place chicken on a plate or tray and season generously with mixed spices
16-season the flour with salt pepper and celery salt
17-coat the chicken pieces in the flour
18-place chicken in hot oil and fry until golden brown
19-fry dark meats t

In [ ]:
with open("Translations.txt", "r") as file:
    strings = file.readlines()

# Remove the newline characters if needed
strings = [s.strip() for s in strings]
add_translation(strings)

In [ ]:
with open('output.json', 'w') as file:
    file.write(data)